In [80]:
## Création d'un DataFrame d'exemple
data = {
    'Name': ['John', 'Anna', 'John', 'Anna', 'John', 'Anna'],
    'Subject': ['Maths', 'Maths', 'Physics', 'Physics', 'Chemistry', 'Chemistry'],
    'Score': [85, 90, 78, 88, 92, 95]
}
 
df = pd.DataFrame(data)
 
## Application de groupby
columns=df.columns
criter = 'Name' 
#howto = 'Subject'
grouped = df.groupby(criter)
for name, group in grouped:
    print("\n", name)
    print(group[[x for x in columns if x!=criter]])


 Anna
     Subject  Score
1      Maths     90
3    Physics     88
5  Chemistry     95

 John
     Subject  Score
0      Maths     85
2    Physics     78
4  Chemistry     92


https://realpython.com/pandas-groupby/

In [81]:
# pandas_legislators.py

import pandas as pd

dtypes = {
    "first_name": "category",
    "gender": "category",
    "type": "category",
    "state": "category",
    "party": "category",
}
df = pd.read_csv(
    "groupby-data/legislators-historical.csv",
    dtype=dtypes,
    usecols=list(dtypes) + ["birthday", "last_name"],
    parse_dates=["birthday"]
)

In [82]:
df

,last_name,first_name,birthday,gender,type,state,party
0,Bassett,Richard,1745-04-02,M,sen,DE,Anti-Administration
1,Bland,Theodorick,1742-03-21,M,rep,VA,NaN
2,Burke,Aedanus,1743-06-16,M,rep,SC,NaN
3,Carroll,Daniel,1730-07-22,M,rep,MD,NaN
4,Clymer,George,1739-03-16,M,rep,PA,NaN
...,...,...,...,...,...,...,...
11970,Garrett,Thomas,1972-03-27,M,rep,VA,Republican
11971,Handel,Karen,1962-04-18,F,rep,GA,Republican
11972,Jones,Brenda,1959-10-24,F,rep,MI,Democrat
11973,Marino,Tom,1952-08-15,M,rep,PA,Republican


In [83]:
df.dtypes

last_name             object
first_name          category
birthday      datetime64[ns]
gender              category
type                category
state               category
party               category
dtype: object

In [84]:
# You call .groupby() and pass the name of the column that you want to group on, 
# which is "state". Then, you use ["last_name"] to specify the columns on which 
# you want to perform the actual aggregation.
n_by_state = df.groupby("state")["last_name"].count()
n_by_state.head(10)

state
AK     16
AL    206
AR    117
AS      2
AZ     48
CA    361
CO     90
CT    240
DC      2
DE     97
Name: last_name, dtype: int64

In [85]:
# Here’s an example of grouping jointly on two columns, which finds the count 
# of Congressional members broken out by state and then by gender:
df.groupby(["state", "gender"])["last_name"].count()

state  gender
AK     F           0
       M          16
AL     F           3
       M         203
AR     F           5
                ... 
WI     M         196
WV     F           1
       M         119
WY     F           2
       M          38
Name: last_name, Length: 116, dtype: int64

In [86]:
n_by_state_gender = df.groupby(["state", "gender"])["last_name"].count()
type(n_by_state_gender)


pandas.core.series.Series

In [87]:
# In the pandas version, the grouped-on columns are pushed into the MultiIndex 
# of the resulting Series by default:
n_by_state_gender.index[:5]

MultiIndex([('AK', 'F'),
            ('AK', 'M'),
            ('AL', 'F'),
            ('AL', 'M'),
            ('AR', 'F')],
           names=['state', 'gender'])

In [88]:
# To more closely emulate the SQL result and push the grouped-on columns back 
# into columns in the result, you can use as_index=False:
# This produces a DataFrame with three columns and a RangeIndex, rather than 
# a Series with a MultiIndex. In short, using as_index=False will make your 
# result more closely mimic the default SQL output for a similar operation.
df.groupby(["state", "gender"], as_index=False)["last_name"].count()


,state,gender,last_name
0,AK,F,0
1,AK,M,16
2,AL,F,3
3,AL,M,203
4,AR,F,5
...,...,...,...
111,WI,M,196
112,WV,F,1
113,WV,M,119
114,WY,F,2


In [89]:
# Don't sort results by the sort keys
df.groupby("state", sort=False)["last_name"].count()

state
DE      97
VA     432
SC     251
MD     305
PA    1053
MA     426
NJ     359
GA     309
NY    1461
NC     354
CT     240
VT     115
KY     373
RI     107
NH     181
TN     299
OH     674
MS     155
OL       2
IN     341
LA     197
IL     486
MO     333
AL     206
AR     117
ME     175
FL     155
MI     294
IA     202
WI     196
TX     256
CA     361
OR      89
MN     160
NM      54
NE     127
WA      95
KS     141
UT      53
NV      56
CO      90
WV     120
DK       9
AZ      48
ID      59
MT      52
WY      40
DC       2
ND      44
SD      51
OK      92
HI      23
PR      19
AK      16
PI      13
VI       4
GU       4
AS       2
Name: last_name, dtype: int64

In [90]:
by_state = df.groupby("state")
print(by_state)

In [91]:
# One useful way to inspect a pandas GroupBy object and see the splitting in action 
# is to iterate over it:
for state, frame in by_state:
    print(f"First 2 entries for {state!r}")
    print("------------------------")
    print(frame.head(2), end="\n\n")

First 2 entries for 'AK'
------------------------
     last_name first_name   birthday gender type state        party
6619    Waskey      Frank 1875-04-20      M  rep    AK     Democrat
6647      Cale     Thomas 1848-09-17      M  rep    AK  Independent

First 2 entries for 'AL'
------------------------
    last_name first_name   birthday gender type state       party
912   Crowell       John 1780-09-18      M  rep    AL  Republican
991    Walker       John 1783-08-12      M  sen    AL  Republican

First 2 entries for 'AR'
------------------------
     last_name first_name   birthday gender type state party
1001     Bates      James 1788-08-25      M  rep    AR   NaN
1279    Conway      Henry 1793-03-18      M  rep    AR   NaN

First 2 entries for 'AS'
------------------------
          last_name first_name   birthday gender type state     party
10797         Sunia       Fofó 1937-03-13      M  rep    AS  Democrat
11755  Faleomavaega        Eni 1943-08-15      M  rep    AS  Democrat

F

In [92]:
# The .groups attribute will give you a dictionary of {group name: group label} pairs. 
# For example, by_state.groups is a dict with states as keys. 
# Here’s the value for the "PA" key:
by_state.groups["PA"]

Int64Index([    4,    19,    21,    27,    38,    57,    69,    76,    84,
               88,
            ...
            11842, 11866, 11875, 11877, 11887, 11891, 11932, 11945, 11959,
            11973],
           dtype='int64', length=1053)

In [93]:
# You can also use .get_group() as a way to drill down to the sub-table from a single group:
by_state.get_group("PA")

,last_name,first_name,birthday,gender,type,state,party
4,Clymer,George,1739-03-16,M,rep,PA,NaN
19,Maclay,William,1737-07-20,M,sen,PA,Anti-Administration
21,Morris,Robert,1734-01-20,M,sen,PA,Pro-Administration
27,Wynkoop,Henry,1737-03-02,M,rep,PA,NaN
38,Jacobs,Israel,1726-06-09,M,rep,PA,NaN
...,...,...,...,...,...,...,...
11891,Brady,Robert,1945-04-07,M,rep,PA,Democrat
11932,Shuster,Bill,1961-01-10,M,rep,PA,Republican
11945,Rothfus,Keith,1962-04-25,M,rep,PA,Republican
11959,Costello,Ryan,1976-09-07,M,rep,PA,Republican


In [94]:
# This is virtually equivalent to using .loc[]
df.loc[df["state"] == "PA"]

,last_name,first_name,birthday,gender,type,state,party
4,Clymer,George,1739-03-16,M,rep,PA,NaN
19,Maclay,William,1737-07-20,M,sen,PA,Anti-Administration
21,Morris,Robert,1734-01-20,M,sen,PA,Pro-Administration
27,Wynkoop,Henry,1737-03-02,M,rep,PA,NaN
38,Jacobs,Israel,1726-06-09,M,rep,PA,NaN
...,...,...,...,...,...,...,...
11891,Brady,Robert,1945-04-07,M,rep,PA,Democrat
11932,Shuster,Bill,1961-01-10,M,rep,PA,Republican
11945,Rothfus,Keith,1962-04-25,M,rep,PA,Republican
11959,Costello,Ryan,1976-09-07,M,rep,PA,Republican


In [95]:
next(iter(by_state))

('AK',
         last_name first_name   birthday gender type state        party
 6619       Waskey      Frank 1875-04-20      M  rep    AK     Democrat
 6647         Cale     Thomas 1848-09-17      M  rep    AK  Independent
 7442      Grigsby     George 1874-12-02      M  rep    AK          NaN
 7501       Sulzer    Charles 1879-02-24      M  rep    AK          NaN
 8039   Sutherland     Daniel 1869-04-17      M  rep    AK   Republican
 8236   Wickersham      James 1857-08-24      M  rep    AK   Republican
 8877       Dimond    Anthony 1881-11-30      M  rep    AK     Democrat
 9819     Gruening     Ernest 1887-02-06      M  sen    AK     Democrat
 9951       Rivers      Ralph 1903-05-23      M  rep    AK     Democrat
 9985     Bartlett     Edward 1904-04-20      M  sen    AK     Democrat
 10082     Pollock     Howard 1920-04-11      M  rep    AK   Republican
 10108      Begich   Nicholas 1932-04-06      M  rep    AK     Democrat
 10325      Gravel    Maurice 1930-05-13      M  sen    A

In [96]:
# From the pandas GroupBy object by_state, you can grab the initial U.S. state 
# and DataFrame with next(). When you iterate over a pandas GroupBy object, 
# you’ll get pairs that you can unpack into two variables:
state, frame = next(iter(by_state))  # First tuple from iterator
state

'AK'

In [107]:
df.groupby("state")["last_name"].count()

state
AK      16
AL     206
AR     117
AS       2
AZ      48
CA     361
CO      90
CT     240
DC       2
DE      97
DK       9
FL     155
GA     309
GU       4
HI      23
IA     202
ID      59
IL     486
IN     341
KS     141
KY     373
LA     197
MA     426
MD     305
ME     175
MI     294
MN     160
MO     333
MS     155
MT      52
NC     354
ND      44
NE     127
NH     181
NJ     359
NM      54
NV      56
NY    1461
OH     674
OK      92
OL       2
OR      89
PA    1053
PI      13
PR      19
RI     107
SC     251
SD      51
TN     299
TX     256
UT      53
VA     432
VI       4
VT     115
WA      95
WI     196
WV     120
WY      40
Name: last_name, dtype: int64

In [106]:
frame["last_name"].count()  # Count for state == 'AK'

16